In [ ]:
import os
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance
from skimage import transform as tf
import random
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Path
input_dir = "/content/drive/MyDrive/data"
output_dir = "/content/drive/MyDrive/data/dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
classes = ["waffle", "beng-beng", "top", "blastoz", "kalpa"]
sample = 500  # total augmentasi

# folder output
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Fungsiaugmentasi
def augmented_image(image_path, output_path, class_name, img_num):
    try:
        img = Image.open(image_path).convert("RGB")

        # rotate
        angle = random.uniform(-25, 25)
        img = img.rotate(angle)

        # flip
        if random.random() > 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
        if random.random() > 0.8:
            img = img.transpose(Image.FLIP_TOP_BOTTOM)

        # zoom
        zoom = random.uniform(0.8, 1.2)
        img = img.resize((int(img.width * zoom), int(img.height * zoom)))

        # shift
        shift_x, shift_y = random.randint(-30, 30), random.randint(-30, 30)
        img = img.transform((img.width, img.height), Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))

        # blur
        if random.random() > 0.7:
            img = img.filter(ImageFilter.GaussianBlur(random.uniform(0.5, 1.5)))

        # brightness
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(random.uniform(0.7, 1.3))

        # contrast
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(random.uniform(0.7, 1.3))

        # crop
        if random.random() > 0.5:
            w, h = img.size
            left = random.randint(0, int(w * 0.2))
            upper = random.randint(0, int(h * 0.2))
            right = w - random.randint(0, int(w * 0.2))
            lower = h - random.randint(0, int(h * 0.2))
            img = img.crop((left, upper, right, lower))
    # save
        img.save(os.path.join(output_path, f"{class_name}_{img_num}.jpg"))
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

In [ ]:
# Loop
for class_name in classes:
    class_input_path = os.path.join(input_dir, class_name)
    class_output_path = os.path.join(output_dir, class_name)
    os.makedirs(class_output_path, exist_ok=True)

    # Process
    for filename in os.listdir(class_input_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(class_input_path, filename)

            # augmentasi
            for i in range(sample):
                augmented_image(image_path, class_output_path, class_name, f"{filename.split('.')[0]}_{i}")